## Count stanzas and lines

In [1]:
from collections import Counter, defaultdict
import itertools
from pathlib import Path
import string

import nltk
import pandas as pd

In [2]:
dst = Path("annotation_tool/tita")

poems = len(list(dst.iterdir()))

stanzas = 0
lines = 0
for e in dst.iterdir():
    st = e.read_text().split("\n\n")[:-1]
    stanzas += len(st)
    for s in st:
        #-1 because the first line is the rhyme code
        lines += (len(s.split("\n"))-1)    


In [3]:
stanzas, lines, poems

(5158, 26198, 508)

In [4]:
poem_df = pd.read_csv(f"tsvs/{dst.name}_rhymes_poems.tsv", sep="\t")
poem_df

,rhyme scheme,stanza,filename,stanza number (in file)
0,AABCCBI,"Gud, la oss i din kunnskap fremmes,\nså det vi...",2009013000065_1_annotated.txt,0
1,TAABCCB,ANNEN SANG\nFra Landego seiles mot Skrova i no...,2007011001018_11_annotated.txt,0
2,AABCCB,Og er det en høstkveld når mørket står på\nså ...,2007011001018_11_annotated.txt,1
3,AABCCB,"Snart ligger og kaien der dyngvåt av regn,\nog...",2007011001018_11_annotated.txt,2
4,AABCCB,Og rett bort i øst ligger Svinøya gård\nhvor B...,2007011001018_11_annotated.txt,3
...,...,...,...,...
5153,AABB,Lukket! Lukket! Øde veier! Gledens vinger brut...,2011041408091_18_annotated.txt,5
5154,AABB,Og nå stod han her som fremmed i et fremmed la...,2011041408091_18_annotated.txt,6
5155,AABB,"Opp av havets blanke bølger, som en vårfrisk m...",2011041408091_18_annotated.txt,7
5156,AABB,"Til Europa! Tanken slår ham plutselig som lyn,...",2011041408091_18_annotated.txt,8


## Count rhyme schemes

In [5]:
rhyme_schemes = list(poem_df["rhyme scheme"])

c = Counter(rhyme_schemes)
l = list(c.items())
l.sort(key = lambda x: x[1], reverse = True)

title_schemes = [s for s in rhyme_schemes if "T" in s]
info_schemes = [s for s in rhyme_schemes if "I" in s]
noise_schemes = [s for s in rhyme_schemes if "N" in s]

print(f"Schemes with title: {len(title_schemes)}\nschemes with info: {len(info_schemes)}\nschemes with noise: {len(noise_schemes)}")
df = pd.DataFrame(l, columns = ["scheme", "count"])
df

Schemes with title: 349
schemes with info: 81
schemes with noise: 2


,scheme,count
0,ABAB,1059
1,ABCB,699
2,AABB,571
3,AABCCB,336
4,ABBA,170
...,...,...
593,ABCDBEC,1
594,ABAAC,1
595,TABBACC,1
596,TIABAC,1


## Remove title, info and noise code for rhyme scheme statistics

In [6]:
new_rhyme_schemes = [scheme.replace("T", "").replace("I", "").replace("N", "") for scheme in rhyme_schemes]
c2 = Counter(new_rhyme_schemes)
l2 = list(c2.items())
l2.sort(key = lambda x: x[1], reverse = True)
df2 = pd.DataFrame(l2, columns = ["scheme", "count"])
df2

,scheme,count
0,ABAB,1142
1,ABCB,749
2,AABB,611
3,AABCCB,360
4,ABBA,181
...,...,...
485,ABCDEFGHG,1
486,ABCDCEA,1
487,ABCDBEC,1
488,ABAAC,1


In [7]:
one_occurence = df2.loc[df2["count"]==1]
one_occurence

,scheme,count
187,AABCDEFD,1
188,ABACBDCD,1
189,ABABBB,1
190,ABBBA,1
191,AABCCDDA,1
...,...,...
485,ABCDEFGHG,1
486,ABCDCEA,1
487,ABCDBEC,1
488,ABAAC,1


In [8]:
top_ten = df2[:10]
top_ten

,scheme,count
0,ABAB,1142
1,ABCB,749
2,AABB,611
3,AABCCB,360
4,ABBA,181
5,AABBCC,162
6,AAA,152
7,ABAAB,73
8,AABCBC,70
9,ABABCC,58


## Create subset for inter-annotator agreement

In [9]:
# Find how many stanzas from each rhyme scheme
top_ten["selection"] = top_ten["count"]/100*2.8
top_ten["selection"] = top_ten["selection"].apply(round)
print(sum(top_ten["selection"]))
#top_ten

100


/tmp/ipykernel_242152/364428577.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_ten["selection"] = top_ten["count"]/100*2.8
/tmp/ipykernel_242152/364428577.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_ten["selection"] = top_ten["selection"].apply(round)


In [10]:
seed = 420

subset = poem_df.loc[poem_df["rhyme scheme"] == ""]

for e in top_ten.itertuples():
    _df = poem_df.loc[poem_df["rhyme scheme"] == e.scheme]
    selection = _df.sample(n=e.selection, random_state=seed)
    print(len(selection))
    subset = pd.concat((subset, selection))

32
21
17
10
5
5
4
2
2
2


In [11]:
subset

,rhyme scheme,stanza,filename,stanza number (in file)
2249,ABAB,"Å, så ramme også meg din vrede.\nLa meg dø! Ta...",2011042608056_9_annotated.txt,12
4752,ABAB,I jubel flyr toget avsted.\nSe grensen! En lin...,2011102708082_46_annotated.txt,1
2471,ABAB,"Sangen har evighet, derfor den skyter\nfortid ...",2006081000055_38_annotated.txt,1
3042,ABAB,Har jeg ikke sett dem de hellige trær\nher i s...,2014102306006_38_annotated.txt,16
2138,ABAB,"Jeg ser hans dag. Jeg ser ham komme,\nden seir...",2011041408047_30_annotated.txt,19
...,...,...,...,...
1082,ABAAB,Dog — nå er hans time inne.\nLyset er til jord...,2011041408091_22_annotated.txt,19
4064,AABCBC,Det har Paiulus ikke glemt.\nDerfor er hans ne...,2011041408091_17_annotated.txt,24
4950,AABCBC,Så til Rom! Det er hans håb forvisst\nalt tilt...,2011041408091_27_annotated.txt,29
4478,ABABCC,Uskyldig! Uskyldig! Jeg gjorde ille!\nTa sølve...,2011042608056_38_annotated.txt,42


## Shuffle subset so not all stanzas with the same scheme come after each other

In [12]:
subset = subset.sample(frac=1, random_state=seed).reset_index(drop=True)
subset

,rhyme scheme,stanza,filename,stanza number (in file)
0,ABAB,"Regn, regn!\nropte skog og mark,\nregn, regn!\...",2006081000055_59_annotated.txt,5
1,ABCB,Vel klager det trette hjerte\ni mismot så mang...,2011041408047_20_annotated.txt,4
2,ABCB,Evig står nå striden\nom den usle grav\nmellom...,2014102306006_110_annotated.txt,5
3,ABAB,"Overalt han har spioner —\nkongen lytter, lure...",2011041408047_21_annotated.txt,1
4,ABCB,"Hva nevnes dog den smerte,\nden sorgens bitre ...",2011042608056_1_annotated.txt,4
...,...,...,...,...
95,ABAB,"I fjellets ly der stein har mæle,\nog dalen åp...",2012032024065_5_annotated.txt,3
96,AABB,Til slutning svevde over vang\nde klare kirkek...,2012032024065_11_annotated.txt,57
97,ABAB,Eller vanens sløve makt\nsom den beste vilje l...,2011041408091_22_annotated.txt,7
98,AABCCB,Eller båter med lina mens leken går vill\nsom ...,2007011001018_7_annotated.txt,6


## Write subset to file

In [13]:
subset_text = "\n\n".join(subset["stanza"])
with open("poem_subset/poem_subset", "w+") as f:
    f.write(subset_text)

##  Total number of rhyme pairs

In [14]:
def no_rhyme(scheme):
    return len(scheme) == len(set(scheme))

def get_edges(vertices):
    return (vertices*(vertices-1))/2

tot_word_pairs = 0
for scheme, count in l2:
    c = Counter(scheme)
    tot_word_pairs += sum(get_edges(v) for v in c.values())*count
    
tot_word_pairs

12631.0

## Number of unique rhyme pairs

In [15]:
punctuation = string.punctuation + "«»—"

def tokenize_and_line_ending_word(line): 
    tokens = [t for t in nltk.tokenize.word_tokenize(line, language='norwegian', preserve_line=False) if t not in punctuation]
    return tokens[-1]
    

pairs = set()

for e in poem_df.itertuples():
    lines = e.stanza.split("\n")
    code = e._1
    d = defaultdict(set)
    for c, line in zip(code, lines):
        if c in ("I", "N", "T"):
            continue
        token = tokenize_and_line_ending_word(line)
        d[c].add(token)
    for s in d.values():
        c = itertools.combinations(s, 2)
        [pairs.add(e) for e in c]

In [16]:
len(pairs)

7421

## Remove mirrored duplicates 

In [17]:
def get_mirrored_dupes(pairs):
    l = list(pairs)
    rem = set()

    for i,(a,b) in enumerate(l):
        if (b,a) in l[i:]:
            rem.add((b, a))
    return rem

rem = get_mirrored_dupes(pairs)

In [18]:
no_dupe_pairs = pairs - rem

In [19]:
print(f"""
Of the {len(pairs)} pairs, there were {len(pairs)-len(no_dupe_pairs)} mirrored duplicates.
Actually unique pairs: {len(no_dupe_pairs)}
""")


Of the 7421 pairs, there were 165 mirrored duplicates.
Actually unique pairs: 7256



In [20]:
pair_df = pd.DataFrame(no_dupe_pairs)
pair_df.to_csv("tsvs/rhyme_pairs.tsv", sep="\t", index=False)

## Create negative samples
We only use the stanzas with the same number of each letter in the rhyme code, as many of the ABCB and similar patterns contain almost-rhymes 

In [21]:
def equal_class_numbers(scheme):
    c = Counter(scheme)
    return len(set(c.values())) == 1

balanced_schemes_df = poem_df.loc[poem_df["rhyme scheme"].apply(equal_class_numbers)]
balanced_schemes_df

,rhyme scheme,stanza,filename,stanza number (in file)
2,AABCCB,Og er det en høstkveld når mørket står på\nså ...,2007011001018_11_annotated.txt,1
3,AABCCB,"Snart ligger og kaien der dyngvåt av regn,\nog...",2007011001018_11_annotated.txt,2
4,AABCCB,Og rett bort i øst ligger Svinøya gård\nhvor B...,2007011001018_11_annotated.txt,3
5,AABCCB,Her fylles nok byen hver landliggedag\nnår fis...,2007011001018_11_annotated.txt,4
6,AABCCB,Her ruller avisen sitt evighetsbånd\nog legger...,2007011001018_11_annotated.txt,5
...,...,...,...,...
5153,AABB,Lukket! Lukket! Øde veier! Gledens vinger brut...,2011041408091_18_annotated.txt,5
5154,AABB,Og nå stod han her som fremmed i et fremmed la...,2011041408091_18_annotated.txt,6
5155,AABB,"Opp av havets blanke bølger, som en vårfrisk m...",2011041408091_18_annotated.txt,7
5156,AABB,"Til Europa! Tanken slår ham plutselig som lyn,...",2011041408091_18_annotated.txt,8


In [22]:
neg_pairs = set()

for e in balanced_schemes_df.itertuples():
    lines = e.stanza.split("\n")
    code = e._1
    d = defaultdict(set)
    for c, line in zip(code, lines):
        token = tokenize_and_line_ending_word(line)
        d[c].add(token)
    
    keys = list(d.keys())
    for i, key in enumerate(keys):
        for j in range(i+1, len(keys)):
            key2 = keys[j]
            for w1 in d[key]:
                for w2 in d[key2]:
                    neg_pairs.add((w1, w2))

In [23]:
rem = get_mirrored_dupes(neg_pairs)
no_dupe_neg_pairs = neg_pairs - rem
print(f"""
Of the {len(neg_pairs)} negative pairs, there were {len(neg_pairs)-len(no_dupe_neg_pairs)} mirrored duplicates.
Actually unique negative pairs: {len(no_dupe_neg_pairs)}
""")


Of the 22591 negative pairs, there were 144 mirrored duplicates.
Actually unique negative pairs: 22447



In [24]:
neg_df = pd.DataFrame(no_dupe_neg_pairs)
neg_df.to_csv("tsvs/negative_rhyme_pairs.tsv", sep="\t", index=False)

## Count unique line ending words

In [25]:
line_ending_words = set()
i = 0
j = 0
for e in poem_df.itertuples():
    lines = e.stanza.split("\n")
    code = e._1
    for c, line in zip(code, lines):
        if c in ("I", "N", "T"):
            j += 1
            continue
        token = tokenize_and_line_ending_word(line)
        line_ending_words.add(token)
        i+=1

In [26]:
len(line_ending_words), i, j, i+j

(7069, 25749, 449, 26198)

In [27]:
print(f"""
The number of unique line ending words in the annotated data set is {len(line_ending_words)}, 
or {(len(line_ending_words)/i)*100}% of the number of potentially rhyming lines.
""")


The number of unique line ending words in the annotated data set is 7069, 
or 27.45349333954717% of the number of potentially rhyming lines.



## Create rhyme annotated sentence pairs

In [29]:
def equal_class_numbers(scheme):
    c = Counter(scheme)
    return len(set(c.values())) == 1

balanced_schemes_df = poem_df.loc[poem_df["rhyme scheme"].apply(equal_class_numbers)]
balanced_schemes_df

,rhyme scheme,stanza,filename,stanza number (in file)
2,AABCCB,Og er det en høstkveld når mørket står på\nså ...,2007011001018_11_annotated.txt,1
3,AABCCB,"Snart ligger og kaien der dyngvåt av regn,\nog...",2007011001018_11_annotated.txt,2
4,AABCCB,Og rett bort i øst ligger Svinøya gård\nhvor B...,2007011001018_11_annotated.txt,3
5,AABCCB,Her fylles nok byen hver landliggedag\nnår fis...,2007011001018_11_annotated.txt,4
6,AABCCB,Her ruller avisen sitt evighetsbånd\nog legger...,2007011001018_11_annotated.txt,5
...,...,...,...,...
5153,AABB,Lukket! Lukket! Øde veier! Gledens vinger brut...,2011041408091_18_annotated.txt,5
5154,AABB,Og nå stod han her som fremmed i et fremmed la...,2011041408091_18_annotated.txt,6
5155,AABB,"Opp av havets blanke bølger, som en vårfrisk m...",2011041408091_18_annotated.txt,7
5156,AABB,"Til Europa! Tanken slår ham plutselig som lyn,...",2011041408091_18_annotated.txt,8


In [30]:
negative = set()

for e in balanced_schemes_df.itertuples():
    lines = e.stanza.split("\n")
    code = e._1
    d = defaultdict(set)
    if "I" in code or "N" in code or "T" in code:
        continue
    for c, line in zip(code, lines):
        d[c].add(line)
    
    # Negative examples = all lines with one letter combined with all lines for all other letters
    keys = list(d.keys())
    for i, key in enumerate(keys):
        for j in range(i+1, len(keys)):
            key2 = keys[j]
            for l1 in d[key]:
                for l2 in d[key2]:
                    negative.add((l1, l2))

In [34]:
neg_df = pd.DataFrame(negative, columns=["sent_a", "sent_b"])
neg_df["rhyme"] = [0]*len(neg_df)
neg_df

,sent_a,sent_b,rhyme
0,«Hvem er feilfri? La oss dømme,taler varmt sin slektnings sak:,0
1,"Med gunstig vind, i brødres lag,","værbitte, snebedekte fjelle.",0
2,"og alle vil hviske om henne, hvis gull","vil følge deg inn i salongen,",0
3,"steg grønne, glade skrenter, fordi hans øyne så.",Det dunkle skaperordet forvandlet han til sang.,0
4,"Jeg hørte den ei skrige,",jeg den ei hørte klynke;,0
...,...,...,...
22835,for gamle folk. De rusler hen,Montro hva fangsten blir idag,0
22836,Og i dette klarhets veld,tvende menn i solglans kommer.,0
22837,"Hans tale var ei viddets glans og skinn,",hans ord var flammen av hans hjertes ild.,0
22838,Den breder seg ut og tar i favn,og Jesrielsletten fyller.,0


In [35]:
positive = set()

for e in poem_df.itertuples():
    lines = e.stanza.split("\n")
    code = e._1
    d = defaultdict(set)
    for c, line in zip(code, lines):
        if c in ("I", "N", "T"):
            continue
        d[c].add(line)
    for s in d.values():
        c = itertools.combinations(s, 2)
        [positive.add(e) for e in c]
        
pos_df = pd.DataFrame(positive, columns=["sent_a", "sent_b"])
pos_df["rhyme"] = [1]*len(pos_df)
pos_df

,sent_a,sent_b,rhyme
0,"Han er eldet, grånet, bøyet,",En av dem med glød i øyet,1
1,og brudgommen trådde sin dans mannhaftig.,da gikk vel hallingen lett og kraftig!,1
2,mens fabrikksirenen uler som så mangen morgen ...,"og maskinene, hvis slave du skal være til du dør.",1
3,deg til konge av en ung og praktfull lediggang...,hvor du bare er en spån i strømmens alltid vil...,1
4,nærmer de seg Romas jord.,Maltas fanger er ombord.,1
...,...,...,...
12566,vånder og vrir seg som kryp i støvet,taler: «Min sjel er til døden bedrøvet»,1
12567,"dagen ut, ja langt på natten med,",siden må med hast han ta avsted.,1
12568,med dette tap som intet kan erstatte.,"Vi må til verdens lys vår stråle skatte,",1
12569,med alle slags utsyn og løfter!,ut mellom de dypskårne grøfter !,1


In [37]:
# concat and shuffle df
sent_df = pd.concat((pos_df, neg_df)).sample(frac=1, random_state=seed)
sent_df

,sent_a,sent_b,rhyme
11586,"i vindens bevende kor,",Jehovas fottrin på jord.,1
8017,Men da sol i det annet gry,"gikk av hav uten mast mot sky,",1
4245,Nå rekker atter den mot lyset opp,med friske knopper i den myke topp:,1
4722,at livet og dets farver var fløyne!,"når de atter slog opp sine øyne,",1
22239,"«Si, hva piner slik ditt sinn?",hen til kongens leie treder:,0
...,...,...,...
266,Hva Norges manndom tapte under dvalen,og fikk sin grøt og logret pent med halen;,1
10394,"Vi eide det, Kariskjæret!","det er som å eie været,",1
1209,"kan ei fortapes, er en hellig arv",som faller renterik til folkets tarv,1
8484,"ser du, Gud, er såre stor,",du for himlen tenner.,0


In [38]:
sent_df.to_csv("tsvs/sentence_pairs.tsv", sep="\t", index=False)